<a href="https://colab.research.google.com/github/Saransh1329/BlackBox-Agentic-AI-for-Predictive-Maintenance/blob/main/ai6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from collections import defaultdict
import warnings
warnings.filterwarnings('ignore')

# Required installations (run these first if needed):
# !pip install pandas openpyxl numpy

class RCACapaAgent:
    """
    Agentic AI for Root Cause Analysis and Corrective/Preventive Action
    Input: Excel file with multiple worksheets
    Output: Same Excel file with new analysis worksheets added
    """

    def __init__(self, excel_path):
        """Initialize the agent with Excel file path"""
        self.excel_path = excel_path
        self.data = {}
        self.analysis = {}
        print("="*70)
        print("🤖 RCA/CAPA AGENTIC AI INITIALIZED")
        print("="*70)

    def load_data(self):
        """Load all worksheets from Excel file"""
        print("\n📂 Loading Excel Data...")
        try:
            excel_file = pd.ExcelFile(self.excel_path)

            sheet_mapping = {
                'owners': ['Owner details', 'Owner Details'],
                'appointments': ['Appointment'],
                'predictions': ['Failiure Prediction', 'Failure Prediction'],
                'maintenance': ['Maintainence Records', 'Maintenance Records'],
                'feedback': ['Feedback'],
                'manufacturer': ['Manufacturer']
            }

            for key, possible_names in sheet_mapping.items():
                for name in possible_names:
                    if name in excel_file.sheet_names:
                        self.data[key] = pd.read_excel(self.excel_path, sheet_name=name)
                        print(f"   ✓ Loaded {name}: {len(self.data[key])} records")
                        break

            print("✅ Data loading complete!")
            return True

        except Exception as e:
            print(f"❌ Error loading data: {str(e)}")
            return False

    def perform_analysis(self):
        """Perform comprehensive RCA/CAPA analysis"""
        print("\n🔍 Performing RCA/CAPA Analysis...")

        vehicle_data = defaultdict(lambda: {
            'predictions': [],
            'maintenance': [],
            'owner': None,
            'feedback': []
        })

        # Organize data by vehicle
        for _, pred in self.data['predictions'].iterrows():
            vid = pred['vehicle_id']
            vehicle_data[vid]['predictions'].append(pred)

        for _, maint in self.data['maintenance'].iterrows():
            vid = maint['vehicle_id']
            vehicle_data[vid]['maintenance'].append(maint)

        for _, owner in self.data['owners'].iterrows():
            vid = owner['vehicle_id']
            vehicle_data[vid]['owner'] = owner

        if 'feedback' in self.data and len(self.data['feedback']) > 0:
            for _, fb in self.data['feedback'].iterrows():
                vid = fb['vehicle_id']
                vehicle_data[vid]['feedback'].append(fb)

        # Perform RCA/CAPA analysis
        rca_results = []
        capa_actions = []
        manufacturing_insights = defaultdict(lambda: {
            'count': 0,
            'vehicles': set(),
            'brands': set(),
            'models': set(),
            'total_cost': 0,
            'rul_hours': [],
            'parts': []
        })

        for vid, vdata in vehicle_data.items():
            if not vdata['predictions']:
                continue

            latest_pred = vdata['predictions'][-1]

            if pd.isna(latest_pred.get('faults_detected')):
                continue

            faults = str(latest_pred['faults_detected']).split(',')
            faults = [f.strip() for f in faults if f.strip()]

            for fault in faults:
                related_maint = [m for m in vdata['maintenance']
                                if not pd.isna(m.get('failure_type')) and
                                fault.lower() in str(m['failure_type']).lower()]

                total_cost = sum([float(m.get('cost_in_inr', 0)) for m in related_maint])

                rca_entry = {
                    'vehicle_id': vid,
                    'fault': fault,
                    'severity': latest_pred.get('highest_severity', 'Unknown'),
                    'failure_prob_pct': float(latest_pred.get('pred_failure_prob_pct', 0)),
                    'rul_hours': latest_pred.get('pred_rul_hours', 'N/A'),
                    'brand': vdata['owner']['brand'] if vdata['owner'] is not None else 'Unknown',
                    'model': vdata['owner']['model'] if vdata['owner'] is not None else 'Unknown',
                    'year': vdata['owner']['year_of_purchase'] if vdata['owner'] is not None else 'Unknown',
                    'maint_count': len(related_maint),
                    'total_cost_inr': round(total_cost, 2),
                    'last_service': related_maint[-1]['maintenance_date'] if related_maint else 'N/A',
                    'parts_replaced': ', '.join([str(m.get('part_replaced', '')) for m in related_maint if m.get('part_replaced')])[:100]
                }

                rca_results.append(rca_entry)

                # Manufacturing insights
                manufacturing_insights[fault]['count'] += 1
                manufacturing_insights[fault]['vehicles'].add(vid)
                if vdata['owner'] is not None:
                    manufacturing_insights[fault]['brands'].add(str(vdata['owner']['brand']))
                    manufacturing_insights[fault]['models'].add(str(vdata['owner']['model']))
                manufacturing_insights[fault]['total_cost'] += total_cost
                if rca_entry['rul_hours'] != 'N/A':
                    try:
                        manufacturing_insights[fault]['rul_hours'].append(float(rca_entry['rul_hours']))
                    except:
                        pass

                # Generate CAPA action
                capa = self.generate_capa_action(rca_entry, related_maint,
                                                manufacturing_insights[fault]['count'])
                capa_actions.append(capa)

        # Sort results
        rca_results.sort(key=lambda x: x['failure_prob_pct'], reverse=True)
        capa_actions.sort(key=lambda x: x['priority_score'], reverse=True)

        # Manufacturing report
        mfg_report = []
        for fault, data in manufacturing_insights.items():
            avg_rul = np.mean(data['rul_hours']) if data['rul_hours'] else None
            quality_impact = 'HIGH' if data['count'] > 10 else 'MEDIUM' if data['count'] > 5 else 'LOW'

            mfg_report.append({
                'fault_type': fault,
                'occurrence_count': data['count'],
                'affected_vehicles': len(data['vehicles']),
                'brands_affected': ', '.join([str(b) for b in data['brands'] if pd.notna(b)])[:100],
                'models_affected': ', '.join([str(m) for m in data['models'] if pd.notna(m)])[:100],
                'total_maintenance_cost_inr': round(data['total_cost'], 2),
                'avg_rul_hours': round(avg_rul, 2) if avg_rul else 'N/A',
                'quality_impact': quality_impact,
                'manufacturing_recommendation': self.get_manufacturing_recommendation(quality_impact, data['count'])
            })

        mfg_report.sort(key=lambda x: x['occurrence_count'], reverse=True)

        # Summary statistics
        summary = {
            'metric': [
                'Total Vehicles Analyzed',
                'Total Faults Detected',
                'Unique Fault Types',
                'High Risk Vehicles (>70% failure prob)',
                'Critical Priority Actions',
                'Total Maintenance Cost (INR)',
                'Critical Quality Issues (HIGH impact)',
                'Analysis Timestamp'
            ],
            'value': [
                len(vehicle_data),
                len(rca_results),
                len(manufacturing_insights),
                len([r for r in rca_results if r['failure_prob_pct'] > 70]),
                len([c for c in capa_actions if c['priority_level'] == 'CRITICAL']),
                f"Rs.{sum([r['total_cost_inr'] for r in rca_results]):.2f}",
                len([m for m in mfg_report if m['quality_impact'] == 'HIGH']),
                datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            ]
        }

        self.analysis = {
            'rca_results': pd.DataFrame(rca_results),
            'capa_actions': pd.DataFrame(capa_actions),
            'manufacturing_report': pd.DataFrame(mfg_report),
            'summary': pd.DataFrame(summary)
        }

        print("✅ Analysis complete!")
        print(f"\n📊 Key Findings:")
        print(f"   • Vehicles analyzed: {len(vehicle_data)}")
        print(f"   • Faults detected: {len(rca_results)}")
        print(f"   • Critical actions: {len([c for c in capa_actions if c['priority_level'] == 'CRITICAL'])}")
        print(f"   • Quality issues: {len([m for m in mfg_report if m['quality_impact'] == 'HIGH'])}")

    def generate_capa_action(self, rca_entry, maintenance_history, pattern_count):
        """Generate CAPA action plan for a specific fault"""
        priority_score = self.calculate_priority_score(rca_entry, pattern_count)

        fault = rca_entry['fault'].lower()
        maint_count = len(maintenance_history)
        failure_prob = rca_entry['failure_prob_pct']
        rul = rca_entry['rul_hours']

        # Comprehensive fault analysis with specific RCA/CAPA for each type

        # ENGINE FAULTS
        if 'engine' in fault or 'motor' in fault:
            if maint_count > 3:
                root_cause = f"RECURRING ENGINE FAILURE: {maint_count} repairs indicate systemic issue. Root causes: (1) Manufacturing defect in engine components, (2) Inadequate quality control, (3) Design flaw in thermal/mechanical stress management, (4) Substandard materials from supplier"
                corrective = f"IMMEDIATE: (1) Complete engine teardown and inspection, (2) Replace all worn components identified, (3) Conduct metallurgical analysis of failed parts, (4) Review service records for pattern identification, (5) Escalate to engineering team for design review"
                preventive = f"LONG-TERM: (1) Implement predictive maintenance using vibration analysis, (2) Oil analysis every 3000km to detect metal particles, (3) Temperature monitoring system installation, (4) Mandatory valve clearance check every 10000km, (5) Driver training on engine stress management, (6) Switch to synthetic oil premium grade"
                mfg_feedback = f"CRITICAL MANUFACTURING ACTION: (1) Audit supplier quality - engine component specifications, (2) Review heat treatment processes, (3) Implement 100% inspection for critical engine parts, (4) Consider design modification for thermal management, (5) Investigate material grade compliance, (6) Root cause investigation with cross-functional team"
            else:
                root_cause = f"ENGINE DEGRADATION: Primary causes: (1) Normal wear and tear accelerated by operating conditions, (2) Inadequate lubrication - oil quality/level issues, (3) Thermal stress from overloading, (4) Possible coolant system inefficiency affecting engine temperature, (5) Carbon buildup reducing efficiency"
                corrective = f"CORRECTIVE: (1) Replace worn engine components: pistons/rings/bearings, (2) Full engine diagnostic with compression test, (3) Oil system flush and premium oil replacement, (4) Valve timing verification and adjustment, (5) Fuel injector cleaning/replacement, (6) Air filter replacement"
                preventive = f"PREVENTIVE: (1) Reduce oil change interval to 5000km, (2) Use manufacturer-recommended synthetic oil, (3) Monthly visual inspection for oil leaks, (4) Quarterly engine performance testing, (5) Avoid prolonged idling and overloading, (6) Install oil pressure monitoring gauge"
                mfg_feedback = f"MANUFACTURING MONITORING: (1) Track engine failure patterns across batches, (2) Review assembly torque specifications, (3) Validate oil pump performance standards, (4) Monitor supplier quality metrics quarterly"

        # BRAKE SYSTEM FAULTS
        elif 'brake' in fault:
            if maint_count > 2:
                root_cause = f"CHRONIC BRAKE ISSUES: Root causes: (1) Inferior brake pad material causing rapid wear, (2) Brake disc quality issues - improper hardness/composition, (3) Caliper malfunction causing uneven pressure distribution, (4) Brake fluid contamination leading to reduced efficiency, (5) Driver behavior - aggressive braking patterns, (6) Design inadequacy for vehicle weight/usage"
                corrective = f"CORRECTIVE: (1) Replace entire brake system: pads, discs, calipers if needed, (2) Complete brake fluid flush with DOT 4 premium fluid, (3) Brake line inspection for corrosion/leaks, (4) Master cylinder pressure test, (5) Wheel bearing inspection (affects brake performance), (6) Brake pedal free play adjustment, (7) Road test for brake balance verification"
                preventive = f"PREVENTIVE: (1) Quarterly brake system inspection, (2) Use OEM or premium aftermarket brake components only, (3) Brake fluid replacement every 12 months, (4) Driver education: proper braking techniques, engine braking usage, (5) Avoid overloading vehicle beyond rated capacity, (6) Install brake temperature sensors for monitoring, (7) Bedding-in procedure after every brake service"
                mfg_feedback = f"CRITICAL QUALITY ISSUE: (1) Review brake component supplier - request quality certifications, (2) Implement incoming inspection for brake parts, (3) Verify friction material composition meets specifications, (4) Investigate brake disc hardness testing, (5) Consider upgrade to better brake system design, (6) Analyze brake failure data across vehicle population"
            else:
                root_cause = f"BRAKE WEAR: Normal causes: (1) Friction material depletion (standard wear), (2) Brake disc surface degradation, (3) Brake fluid hygroscopic absorption reducing boiling point, (4) Environmental factors - dust, moisture, (5) Driving conditions - urban stop-and-go traffic"
                corrective = f"CORRECTIVE: (1) Replace brake pads (minimum 3mm remaining thickness), (2) Resurface or replace brake discs if warped/scored, (3) Brake fluid flush and replacement, (4) Caliper slide pin lubrication, (5) Brake hardware cleaning and anti-seize application"
                preventive = f"PREVENTIVE: (1) Brake inspection every 5000km or 6 months, (2) Avoid riding brake pedal, (3) Use engine braking on descents, (4) Maintain proper tire pressure (affects braking), (5) Brake fluid change annually, (6) Visual check for brake pad thickness monthly"
                mfg_feedback = f"STANDARD MONITORING: (1) Track brake component lifespan statistics, (2) Review if failure rate exceeds industry standards, (3) Document brake performance feedback from service centers"

        # BATTERY/ELECTRICAL FAULTS
        elif 'battery' in fault or 'alternator' in fault or 'electrical' in fault:
            root_cause = f"ELECTRICAL SYSTEM FAILURE: Root causes: (1) Battery aging - sulfation and capacity loss beyond 80%, (2) Alternator failure - diode/regulator malfunction causing improper charging, (3) Parasitic drain - electrical components drawing power when off, (4) Loose/corroded battery terminals increasing resistance, (5) Voltage regulator malfunction causing overcharging/undercharging, (6) Belt tension issues affecting alternator performance"
            corrective = f"CORRECTIVE: (1) Battery load test and replacement if CCA below 80% of rating, (2) Alternator output test - verify 13.5-14.5V at idle and load conditions, (3) Parasitic draw test - should be under 50mA with vehicle off, (4) Battery terminal cleaning and anti-corrosion treatment, (5) Alternator belt tension check and replacement if worn, (6) Voltage regulator testing, (7) Wiring harness inspection for damaged insulation, (8) Ground connection verification"
            preventive = f"PREVENTIVE: (1) Monthly battery voltage check (12.6V+ when off, 13.5-14.5V when running), (2) Battery terminal cleaning and tightening quarterly, (3) Avoid deep discharge cycles - use battery tender if vehicle stored, (4) Limit short trips (prevents full recharge), (5) Inspect alternator belt every 10000km, (6) Keep battery top clean and dry, (7) Test charging system annually, (8) Upgrade to AGM battery if frequent failures"
            mfg_feedback = f"MANUFACTURING REVIEW: (1) Audit battery supplier quality - check warranty claims rate, (2) Review alternator specifications for adequate capacity, (3) Verify voltage regulator calibration in production, (4) Investigate wiring harness routing for chafing points, (5) Consider battery monitoring system integration, (6) Analyze electrical system design for parasitic draw sources"

        # COOLING SYSTEM FAULTS
        elif 'coolant' in fault or 'temperature' in fault or 'thermostat' in fault or 'radiator' in fault or 'cooling' in fault:
            root_cause = f"THERMAL MANAGEMENT FAILURE: Root causes: (1) Thermostat failure - stuck open/closed preventing proper temperature regulation, (2) Radiator blockage - internal/external reducing coolant flow, (3) Water pump impeller wear/failure reducing circulation, (4) Coolant degradation - loss of anti-corrosion additives, (5) Air pockets in cooling system, (6) Radiator cap failure - incorrect pressure maintenance, (7) Cooling fan malfunction, (8) Head gasket leak introducing combustion gases"
            corrective = f"CORRECTIVE: (1) Thermostat replacement with OEM spec (verify opening temperature), (2) Radiator flush - remove scale and debris, (3) Pressure test cooling system to 1.1 bar for leaks, (4) Water pump inspection - check for play and seal leaks, (5) Coolant replacement with 50/50 mix ethylene glycol, (6) Radiator cap replacement (verify pressure rating), (7) Cooling fan operation test, (8) Hose inspection and replacement if cracked/soft, (9) Heater core flush, (10) Head gasket combustion leak test"
            preventive = f"PREVENTIVE: (1) Coolant flush and replacement every 2 years or 40000km, (2) Monthly coolant level check when cold, (3) Visual inspection for leaks and hose condition, (4) Radiator external cleaning (remove bugs/debris), (5) Monitor temperature gauge for abnormal readings, (6) Avoid overloading in hot weather, (7) Let engine warm up before high load, (8) Park in shade when possible, (9) Annual pressure test of cooling system"
            mfg_feedback = f"DESIGN/QUALITY REVIEW: (1) Evaluate radiator capacity for vehicle power output and climate, (2) Review thermostat supplier quality and failure rates, (3) Verify water pump design for durability, (4) Consider auxiliary cooling fan for hot climates, (5) Investigate coolant specification compatibility with engine materials, (6) Review cooling system routing for air pocket prevention"

        # OIL PRESSURE/LUBRICATION FAULTS
        elif 'oil' in fault or 'lubrication' in fault or 'pressure' in fault:
            root_cause = f"LUBRICATION SYSTEM FAILURE: Root causes: (1) Oil pump wear - reduced pressure generation capacity, (2) Oil viscosity breakdown - thermal/oxidative degradation, (3) Oil filter clogging - restricted flow, (4) Internal engine wear - bearing clearances excessive, (5) Oil level low - insufficient supply, (6) Pressure relief valve malfunction, (7) Oil passages blocked with sludge, (8) Wrong oil grade for operating conditions"
            corrective = f"CORRECTIVE: (1) Oil pressure test with mechanical gauge (verify against specifications), (2) Oil pump replacement if pressure below spec, (3) Complete oil change with correct viscosity grade, (4) New OEM oil filter installation, (5) Engine flush to remove sludge (if excessive), (6) Bearing clearance inspection (may require teardown), (7) Pressure relief valve cleaning/replacement, (8) Oil passage inspection and cleaning, (9) PCV system check for proper crankcase ventilation"
            preventive = f"PREVENTIVE: (1) Reduce oil change interval to 5000km, (2) Use high-quality synthetic oil meeting manufacturer specs, (3) Weekly oil level check and top-up, (4) Oil analysis every 10000km (check for metal particles, viscosity), (5) Monitor oil pressure gauge for abnormal readings, (6) Avoid extended high-RPM operation, (7) Always use OEM or equivalent oil filters, (8) Address oil leaks immediately, (9) Warm up engine before driving"
            mfg_feedback = f"MANUFACTURING INVESTIGATION: (1) Review oil pump quality control and testing procedures, (2) Verify oil passage design and drilling quality, (3) Investigate bearing supplier quality and clearances, (4) Review assembly procedures for cleanliness, (5) Consider oil pressure monitoring system as standard, (6) Analyze oil consumption patterns across fleet"

        # FUEL SYSTEM FAULTS
        elif 'fuel' in fault or 'injector' in fault or 'pump' in fault:
            root_cause = f"FUEL SYSTEM ISSUES: Root causes: (1) Fuel pump wear - reduced pressure delivery, (2) Fuel filter clogging - contamination restricting flow, (3) Injector fouling - carbon deposits affecting spray pattern, (4) Fuel quality issues - water/particulate contamination, (5) Fuel pressure regulator malfunction, (6) Fuel line leaks/restrictions, (7) Tank contamination"
            corrective = f"CORRECTIVE: (1) Fuel pressure test (verify against spec PSI), (2) Fuel pump replacement if pressure inadequate, (3) Fuel filter replacement, (4) Injector cleaning service or replacement, (5) Fuel system cleaning additive treatment, (6) Fuel pressure regulator testing/replacement, (7) Fuel line inspection for leaks/kinks, (8) Fuel tank inspection and cleaning if contaminated"
            preventive = f"PREVENTIVE: (1) Use high-quality fuel from reputable stations, (2) Fuel filter replacement every 20000km, (3) Add fuel system cleaner every 5000km, (4) Keep fuel tank above 1/4 full (prevents pump overheating), (5) Replace fuel cap if seal damaged, (6) Avoid filling tank when tanker is refilling station (sediment), (7) Annual fuel system inspection"
            mfg_feedback = f"QUALITY MONITORING: (1) Track fuel system component failure rates, (2) Review fuel pump supplier quality, (3) Verify fuel filter specifications adequate, (4) Consider fuel quality sensor integration, (5) Review fuel tank design for contamination prevention"

        # TRANSMISSION FAULTS
        elif 'transmission' in fault or 'gearbox' in fault or 'clutch' in fault:
            root_cause = f"TRANSMISSION FAILURE: Root causes: (1) Transmission fluid degradation - loss of lubrication/friction properties, (2) Clutch wear - friction material depletion, (3) Synchronizer wear causing difficult shifts, (4) Hydraulic system issues (if automatic), (5) Torque converter failure, (6) Gear tooth wear from inadequate lubrication, (7) Driver behavior - aggressive shifting"
            corrective = f"CORRECTIVE: (1) Transmission fluid drain and refill with correct spec fluid, (2) Clutch inspection and replacement if slipping, (3) Transmission diagnostic scan for codes, (4) Synchronizer inspection and replacement if worn, (5) Hydraulic system pressure test, (6) Shift linkage adjustment, (7) Flywheel resurfacing if needed"
            preventive = f"PREVENTIVE: (1) Transmission fluid change every 40000km, (2) Avoid resting foot on clutch pedal, (3) Complete clutch engagement before acceleration, (4) Let transmission warm up before hard driving, (5) Regular transmission fluid level check, (6) Avoid towing beyond rated capacity, (7) Use correct driving technique for conditions"
            mfg_feedback = f"DESIGN REVIEW: (1) Review clutch material specifications, (2) Verify transmission fluid capacity adequate, (3) Investigate cooling requirements, (4) Consider transmission temperature monitoring, (5) Review shift mechanism design"

        # SUSPENSION/STEERING FAULTS
        elif 'suspension' in fault or 'steering' in fault or 'shock' in fault or 'strut' in fault:
            root_cause = f"SUSPENSION SYSTEM ISSUES: Root causes: (1) Shock absorber fluid leakage - loss of dampening, (2) Bushing wear - rubber deterioration, (3) Ball joint wear - increased play, (4) Tie rod end wear affecting steering, (5) Spring fatigue/breakage, (6) Strut mount bearing failure, (7) Overloading causing premature wear"
            corrective = f"CORRECTIVE: (1) Replace worn shocks/struts (both sides), (2) Ball joint replacement if excessive play, (3) Bushing replacement with polyurethane upgrades, (4) Tie rod end replacement, (5) Wheel alignment after repairs, (6) Spring replacement if sagged/broken, (7) Strut mount bearing replacement"
            preventive = f"PREVENTIVE: (1) Avoid potholes and rough roads when possible, (2) Regular wheel alignment every 20000km, (3) Suspension inspection every 10000km, (4) Maintain proper tire pressure, (5) Avoid overloading vehicle, (6) Check shock absorber condition annually, (7) Lubricate grease fittings if equipped"
            mfg_feedback = f"QUALITY REVIEW: (1) Review shock absorber supplier quality and warranty rates, (2) Verify suspension design for road conditions, (3) Investigate bushing material durability, (4) Consider upgraded components for harsh conditions"

        # EXHAUST SYSTEM FAULTS
        elif 'exhaust' in fault or 'emission' in fault or 'catalyst' in fault:
            root_cause = f"EXHAUST SYSTEM FAILURE: Root causes: (1) Catalytic converter substrate melting from overheating, (2) Oxygen sensor failure affecting fuel mixture, (3) Exhaust manifold crack from thermal stress, (4) Muffler corrosion from moisture/road salt, (5) EGR valve carbon buildup, (6) Exhaust leak causing backpressure issues"
            corrective = f"CORRECTIVE: (1) Replace faulty oxygen sensors, (2) Catalytic converter replacement if substrate damaged, (3) Exhaust manifold repair/replacement, (4) Muffler and pipe replacement if corroded, (5) EGR valve cleaning or replacement, (6) Exhaust leak repair with new gaskets"
            preventive = f"PREVENTIVE: (1) Annual exhaust system inspection, (2) Address check engine lights promptly, (3) Use fuel additives to reduce carbon buildup, (4) Let engine warm up fully, (5) Avoid short trips (causes moisture buildup), (6) Apply anti-rust coating to exhaust components"
            mfg_feedback = f"DESIGN CONSIDERATIONS: (1) Review catalytic converter positioning for heat management, (2) Verify exhaust gas temperature specifications, (3) Investigate corrosion-resistant coating options, (4) Consider upgraded oxygen sensor specifications"

        # TIRE/WHEEL FAULTS
        elif 'tire' in fault or 'tyre' in fault or 'wheel' in fault:
            root_cause = f"TIRE ISSUES: Root causes: (1) Improper tire pressure - under/over inflation, (2) Uneven wear from misalignment, (3) Tire age and rubber degradation, (4) Road hazard damage, (5) Wheel bearing wear, (6) Improper tire selection for conditions"
            corrective = f"CORRECTIVE: (1) Replace worn/damaged tires, (2) Wheel alignment to specifications, (3) Wheel balancing, (4) Tire pressure adjustment to door placard specs, (5) Wheel bearing replacement if noisy, (6) Tire rotation"
            preventive = f"PREVENTIVE: (1) Check tire pressure monthly including spare, (2) Tire rotation every 8000km, (3) Wheel alignment every 20000km or if vehicle pulls, (4) Visual tire inspection for cuts/bulges weekly, (5) Replace tires at 2/32\" tread depth, (6) Store seasonal tires properly, (7) Avoid potholes and curbs"
            mfg_feedback = f"SPECIFICATION REVIEW: (1) Verify recommended tire specifications appropriate, (2) Review wheel bearing quality, (3) Consider tire pressure monitoring system, (4) Document tire wear patterns for design feedback"

        # GENERIC/OTHER FAULTS
        else:
            root_cause = f"COMPONENT FAILURE DETECTED: Root causes: (1) Normal component wear beyond service life, (2) Operating conditions exceeding design parameters, (3) Possible manufacturing defect - {failure_prob:.1f}% failure probability detected, (4) Maintenance interval exceeded, (5) Environmental factors accelerating degradation, (6) {maint_count} previous repair(s) indicate recurring issue"
            corrective = f"CORRECTIVE: (1) Complete diagnostic scan to identify specific failure mode, (2) Replace affected component with OEM part, (3) Inspect related systems for secondary damage, (4) Review maintenance history for patterns, (5) Perform functional test after repair, (6) Document failure mode for engineering review"
            preventive = f"PREVENTIVE: (1) Implement condition monitoring for this component, (2) Reduce maintenance interval by 20%, (3) Use premium replacement parts, (4) Conduct periodic inspections at {int(float(rul) * 0.5) if rul != 'N/A' else 5000} hour intervals, (5) Monitor for early warning signs, (6) Train operators on proper usage"
            mfg_feedback = f"INVESTIGATION REQUIRED: (1) Collect failed component for engineering analysis, (2) Review design specifications and tolerances, (3) Audit supplier quality control, (4) Investigate if failure pattern exists across fleet, (5) Consider design improvement or component upgrade"

        timeline = "IMMEDIATE (24-48 hours)" if failure_prob > 80 else \
                  "URGENT (Within 1 week)" if failure_prob > 60 else \
                  "SCHEDULED (Within 2-4 weeks)"

        return {
            'vehicle_id': rca_entry['vehicle_id'],
            'fault': rca_entry['fault'],
            'brand': rca_entry['brand'],
            'model': rca_entry['model'],
            'year': rca_entry['year'],
            'priority_score': round(priority_score, 2),
            'priority_level': 'CRITICAL' if priority_score > 80 else
                            'HIGH' if priority_score > 60 else
                            'MEDIUM' if priority_score > 40 else 'LOW',
            'failure_prob_pct': f"{rca_entry['failure_prob_pct']:.1f}%",
            'rul_hours': rca_entry['rul_hours'],
            'root_cause_analysis': root_cause,
            'corrective_action': corrective,
            'preventive_action': preventive,
            'recommended_timeline': timeline,
            'manufacturing_feedback': mfg_feedback,
            'maintenance_cost_inr': rca_entry['total_cost_inr'],
            'service_count': rca_entry['maint_count']
        }

    def calculate_priority_score(self, rca_entry, pattern_count):
        """Calculate priority score for CAPA action"""
        score = 0
        score += rca_entry['failure_prob_pct'] * 0.5
        score += rca_entry['maint_count'] * 8
        score += pattern_count * 3

        if rca_entry['severity'] == 'Critical':
            score += 25
        elif rca_entry['severity'] == 'Warning':
            score += 15

        if rca_entry['rul_hours'] != 'N/A':
            try:
                if float(rca_entry['rul_hours']) < 100:
                    score += 20
            except:
                pass

        return min(score, 100)

    def get_manufacturing_recommendation(self, impact, count):
        """Generate manufacturing recommendation"""
        if impact == 'HIGH':
            return f"URGENT: Conduct design review and quality audit. {count} occurrences indicate systemic issue. Review supplier quality and specifications."
        elif impact == 'MEDIUM':
            return "Monitor closely. Implement enhanced quality checks during manufacturing. Consider preventive design modifications."
        else:
            return "Normal monitoring. Document pattern for future quality improvements."

    def save_analysis_to_excel(self, output_path=None):
        """Save analysis results to Excel file with new worksheets"""
        print("\n💾 Saving Analysis to Excel...")

        if output_path is None:
            output_path = self.excel_path.replace('.xlsx', '_RCA_CAPA_Analysis.xlsx')

        try:
            # Read original Excel file
            with pd.ExcelFile(self.excel_path) as xls:
                original_sheets = {sheet_name: pd.read_excel(xls, sheet_name)
                                 for sheet_name in xls.sheet_names}

            # Create Excel writer
            with pd.ExcelWriter(output_path, engine='openpyxl') as writer:
                # Write original sheets
                for sheet_name, df in original_sheets.items():
                    df.to_excel(writer, sheet_name=sheet_name, index=False)

                # Write analysis sheets
                self.analysis['summary'].to_excel(writer, sheet_name='RCA_Summary', index=False)
                self.analysis['rca_results'].to_excel(writer, sheet_name='RCA_Results', index=False)
                self.analysis['capa_actions'].to_excel(writer, sheet_name='CAPA_Actions', index=False)
                self.analysis['manufacturing_report'].to_excel(writer, sheet_name='Manufacturing_Insights', index=False)

                # Auto-adjust column widths
                for sheet_name in ['RCA_Summary', 'RCA_Results', 'CAPA_Actions', 'Manufacturing_Insights']:
                    worksheet = writer.sheets[sheet_name]
                    for column in worksheet.columns:
                        max_length = 0
                        column = [cell for cell in column]
                        for cell in column:
                            try:
                                if len(str(cell.value)) > max_length:
                                    max_length = len(cell.value)
                            except:
                                pass
                        adjusted_width = min(max_length + 2, 50)
                        worksheet.column_dimensions[column[0].column_letter].width = adjusted_width

            print(f"✅ Analysis saved to: {output_path}")
            print("\n📋 New Worksheets Added:")
            print("   • RCA_Summary - Executive summary and key metrics")
            print("   • RCA_Results - Detailed root cause analysis for each vehicle/fault")
            print("   • CAPA_Actions - Corrective and preventive action plans")
            print("   • Manufacturing_Insights - Quality issues and recommendations for manufacturing team")

            return output_path

        except Exception as e:
            print(f"❌ Error saving analysis: {str(e)}")
            return None

    def run(self, output_path=None):
        """Execute complete RCA/CAPA analysis pipeline"""
        print("\n🚀 Starting RCA/CAPA Agentic AI Pipeline...")

        if not self.load_data():
            return None

        self.perform_analysis()
        result_path = self.save_analysis_to_excel(output_path)

        print("\n" + "="*70)
        print("✅ RCA/CAPA ANALYSIS COMPLETE!")
        print("="*70)

        return result_path


# USAGE EXAMPLE - FILE UPLOAD VERSION
if __name__ == "__main__":
    print("\n" + "="*70)
    print("🤖 RCA/CAPA AGENTIC AI - UPLOAD YOUR FILE")
    print("="*70)

    # For Jupyter Notebook / Colab - File Upload Widget
    try:
        from google.colab import files
        print("\n📤 Please upload your Excel file...")
        uploaded = files.upload()

        if uploaded:
            excel_file_path = list(uploaded.keys())[0]
            print(f"✅ File uploaded: {excel_file_path}")

            # Initialize agent with uploaded file
            agent = RCACapaAgent(excel_file_path)

            # Run complete analysis
            output_file = agent.run()

            if output_file:
                print(f"\n🎉 Analysis complete!")
                print(f"\n📥 Downloading result file...")
                files.download(output_file)
            else:
                print("\n❌ Analysis failed. Please check the errors above.")
        else:
            print("❌ No file uploaded.")

    except ImportError:
        # For regular Jupyter Notebook (not Colab)
        try:
            from ipywidgets import FileUpload
            from IPython.display import display
            import io

            print("\n📤 Click the button below to upload your Excel file...")

            uploader = FileUpload(accept='.xlsx,.xls', multiple=False)
            display(uploader)

            # Note: After upload, run this separately:
            print("\n⚠️ After uploading, run this code:")
            print("""
# Get uploaded file
uploaded_file = uploader.value[0]
filename = uploaded_file['name']

# Save to local file
with open(filename, 'wb') as f:
    f.write(uploaded_file['content'])

# Run analysis
agent = RCACapaAgent(filename)
output_file = agent.run()
print(f"✅ Analysis saved to: {output_file}")
""")

        except ImportError:
            print("\n⚠️ File upload widget not available.")
            print("Please use one of these methods:")
            print("\n1. Direct file path:")
            print("   agent = RCACapaAgent('your_file.xlsx')")
            print("   output_file = agent.run()")
            print("\n2. Or run in Google Colab for automatic file upload")

    except Exception as e:
        print(f"\n❌ Error: {str(e)}")
        print("\nAlternative: Use direct file path")
        print("agent = RCACapaAgent('your_file.xlsx')")
        print("output_file = agent.run()")


🤖 RCA/CAPA AGENTIC AI - UPLOAD YOUR FILE

📤 Please upload your Excel file...


Saving scheduled_appointments (1) (2).xlsx to scheduled_appointments (1) (2) (2).xlsx
✅ File uploaded: scheduled_appointments (1) (2) (2).xlsx
🤖 RCA/CAPA AGENTIC AI INITIALIZED

🚀 Starting RCA/CAPA Agentic AI Pipeline...

📂 Loading Excel Data...
   ✓ Loaded Owner Details: 1970 records
   ✓ Loaded Appointment: 1120 records
   ✓ Loaded Failiure Prediction: 1970 records
   ✓ Loaded Maintainence Records: 7890 records
   ✓ Loaded Feedback: 389 records
✅ Data loading complete!

🔍 Performing RCA/CAPA Analysis...
✅ Analysis complete!

📊 Key Findings:
   • Vehicles analyzed: 3456
   • Faults detected: 1665
   • Critical actions: 1364
   • Quality issues: 19

💾 Saving Analysis to Excel...
✅ Analysis saved to: scheduled_appointments (1) (2) (2)_RCA_CAPA_Analysis.xlsx

📋 New Worksheets Added:
   • RCA_Summary - Executive summary and key metrics
   • RCA_Results - Detailed root cause analysis for each vehicle/fault
   • CAPA_Actions - Corrective and preventive action plans
   • Manufacturing_Insigh

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>